In [1]:
import sys
sys.path.append('..')  # 将父文件夹添加到 Python 路径中，不然找不到下面的包

from utilities.set_env import set_env
from utilities.tushare_limit_up import *

In [2]:
set_env()

instance = TushareLimitUp()
cal_date = instance.getLatestTradeData()

cal_date

'20240315'

limit_list_d接口获取到的数据是收盘价等于涨停板的股票
daily中进行筛选获取涨幅大于10%的股票
两者取股票代码的并集

然后对基础的数据进行合并

In [6]:
df = instance.generate_limit_up_df()

df[:5]

,ts_code,name,industry,close,pct_chg,amount,circ_mv,total_mv,pb,pe,turnover_rate,turnover_rate_f,limit_amount,fd_amount,first_time,last_time,open_times,up_stat,limit_times
73,603580.SH,艾艾精工,塑料,22.28,10.0247,2.910304,29.113989,29.113989,6.6030,NaN,10.0032,30.2525,NaN,0.824532,93018,141748,2.0,9/9,9.0
12,002076.SZ,星光股份,家用电器,2.90,9.8485,6.802007,30.062564,32.164610,10.0976,130.7021,23.1698,26.2392,NaN,3.348299,94327,94327,0.0,5/5,5.0
77,603963.SH,大理药业,中成药,10.91,9.9798,1.800509,23.969270,23.969270,6.2134,NaN,7.7949,26.9386,NaN,0.286624,93133,93312,2.0,4/4,4.0
76,603879.SH,永悦科技,化工原料,7.15,10.0000,2.458839,25.693127,25.791619,5.5049,NaN,9.5760,15.5434,NaN,0.591070,92502,95620,1.0,4/4,4.0
64,603095.SH,越剑智能,纺织机械,18.62,9.9823,2.054350,34.409760,34.409760,1.3707,7.6449,6.2345,22.0041,NaN,0.397240,102932,102932,0.0,2/2,2.0


In [7]:
from utilities.tushare_limit_up import generate_limit_up_excel
path = generate_limit_up_excel(df, cal_date)

import os
os.startfile(path)

'./涨停分析-20240315.xlsx'